# 1. Data import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visiualization
import seaborn as sns # data visisualization like distribytion chart, matrix plot, heat maps
import sklearn # scikit library for machine learning

###########

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')
sample_submission=pd.read_csv('../input/quora-insincere-questions-classification/sample_submission.csv')

   # 2.Descriptives

In [ ]:
print(train.columns)
print(train.info)
print(train.head())
print(train.describe())

## Exploratory data analysis

In [ ]:
train['question_text'].str.len().hist() #number of characters present in each sentence

### Number of words appearing in each line.

In [ ]:
train['question_text'].str.split().map(lambda x: len(x)).hist()

### Average word length in each sentence

In [ ]:
train['question_text'].str.split().apply(lambda x : [len(i) for i in x]). map(lambda x: np.mean(x)).hist() 

## Measuring insincere and sincere questions

In [ ]:
insincere=train[train['target']==1]

sincere=train[train['target']==0]

print("First 10 samples of insincere questions\n".format(),insincere[:10])

print("First 10 samples of sincere \n".format(),sincere[:10])

## Frequency count of sincere and insincere question texts:

In [ ]:
count=train['target'].value_counts()

print('Total Counts of both sets'.format(),count)

print("==============")

#Creating a function to plot the counts using matplotlib
def plot_counts(count_good,count_bad):
    plt.rcParams['figure.figsize']=(6,6)
    plt.bar(0,count_good,width=0.6,
            label='Sincere Questions',
            color='Indigo')
    plt.legend()
    plt.bar(2,count_bad,width=0.6,
            label='Insincere Questions',
            color='Red')
    plt.legend()
    plt.ylabel('Count of Questions')
    plt.xlabel('Types of Questions')
    plt.show()

In [ ]:
count_good=train[train['target']==0]
count_bad=train[train['target']==1]
plot_counts(len(count_good),len(count_bad))

## Analyse the count of words in each segment- both positive and negative reviews

In [ ]:
#Function for checking word length
def cal_len(data):
    return len(data)

#Create generic plotter with Seaborn
def plot_count(count_ones,count_zeros,title_1,title_2,subtitle):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
    sns.distplot(count_zeros,ax=ax1,color='Blue')
    ax1.set_title(title_1)
    sns.distplot(count_ones,ax=ax2,color='Red')
    ax2.set_title(title_2)
    fig.suptitle(subtitle)
    plt.show()   

In [ ]:
count_sincere=count_good['question_text'].str.split().apply(lambda z:cal_len(z))
count_insincere=count_bad['question_text'].str.split().apply(lambda z:cal_len(z))
print("Sincere Questions:" + str(count_sincere))
print("Insincere Questions:" + str(count_insincere))
plot_count(count_sincere,count_insincere,"Sincere Questions",
           "Insincere Questions","Questions Analysis")

## Count Punctuations/Stopwords/Codes and other semantic datatypes

In [ ]:
#We will be using the "generic_plotter" function.

count_sincere_punctuations=count_good['question_text'].apply(lambda z: len([c for c in str(z) if c in string.punctuation]))

count_insincere_punctuations=count_bad['question_text'].apply(lambda z:len([c for c in str(z) if c in string.punctuation]))

plot_count(count_sincere_punctuations,count_insincere_punctuations,"Sincere Questions Punctuations","Insincere Questions Punctuations","Questions Punctuation Analysis")

## Analyse Stopwords

In [ ]:
def plot_count_1(count_ones,count_zeros,title_1,title_2,subtitle):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
    sns.distplot(count_zeros,ax=ax1,color='Blue')
    ax1.set_title(title_1)
    sns.distplot(count_ones,ax=ax2,color='Orange')
    ax2.set_title(title_2)
    fig.suptitle(subtitle)
    plt.show()    

In [ ]:
stops=set(stopwords.words('english'))

count_sincere_stops=count_good['question_text'].apply(lambda z : np.mean([len(z) for w in str(z).split()]))

count_insincere_stops=count_bad['question_text'].apply(lambda z : np.mean([len(z) for w in str(z).split()]))

plot_count_1(count_sincere_stops,count_insincere_stops,"Sincere Questions Stopwords","Insincere Questions Stopwords","Questions Stopwords Analysis")

# Visualization of most occuring words through word cloud

In [ ]:
def display_cloud(data,color,figsize):
    plt.subplots(figsize=figsize)
    wc = WordCloud(stopwords=STOPWORDS,
                   background_color="white", 
                   contour_width=2, 
                   contour_color=color,
                   max_words=2000, 
                   max_font_size=256,
                   random_state=42)
    wc.generate(' '.join(data))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis('off')
    plt.show()

In [ ]:
display_cloud(train['question_text'],
              color='red',
              figsize=(5,5)) #WordCloud for the insincere questions

In [ ]:
display_cloud(sincere['question_text'],
              color='red',
              figsize=(5,5)) #WordCloud for the sincere questions

In [ ]:
display_cloud(insincere['question_text'],
              color='red',
              figsize=(5,5)) #WordCloud for the insincere questions

## Building corpus

In [ ]:
import nltk
nltk.download('stopwords')
stop=set(stopwords.words('english'))

In [ ]:
#create the corpus.

corpus=[]
question_text = train['question_text'].str.split()
question_text= question_text.values.tolist()
corpus=[word for i in question_text for word in i]

In [ ]:
from collections import defaultdict
dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1

## occurrences of each word in a list of tuples

In [ ]:
counter= Counter(corpus)
most = counter.most_common()

x, y= [], []
for word,count in most[:40]:
    if (word not in stop):
        x.append(word)
        y.append(count)
        
sns.barplot(x=y,y=x)

## Inference From Exploratory Analysis

The following can be inferred from the data:

The dataset is imbalanced.
The dataset contains redundant words and html syntaxes.
Punctuations/stopwords are present in a equal distribution in the dataset.
This tells us that we have to do lots of cleaning!

## N-grams

In [ ]:
#Ngram exploration
from nltk.util import ngrams

In [ ]:
def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:10]

## bigrams

In [ ]:
insincere_label=train[train['target']== 1]['question_text']
print(insincere_label.head())  #top insincere questions


sincere_label=train[train['target']== 0]['question_text']
print(sincere_label.head())  #top sincere questions

Bigram Insincere Questions

In [ ]:
bigram_insincere=get_top_ngram(insincere_label,n=2)
x,y=map(list,zip(*bigram_insincere))
sns.barplot(x=y,y=x)

Bigram sincere questions

In [ ]:
bigram_sincere=get_top_ngram(sincere_label,n=2)
x,y=map(list,zip(*bigram_sincere))
sns.barplot(x=y,y=x)

## Trigrams

Trigram insincere questions

In [ ]:
trigram_insincere=get_top_ngram(insincere_label,n=3)
x,y=map(list,zip(*trigram_insincere))
sns.barplot(x=y,y=x)

Trigram sincere questions

In [ ]:
trigram_sincere=get_top_ngram(sincere_label,n=3)
x,y=map(list,zip(*trigram_sincere))
sns.barplot(x=y,y=x)

## Pentagram

Pentagram insincere questions

In [ ]:
pentagram_insincere=get_top_ngram(insincere_label,n=5)
x,y=map(list,zip(*pentagram_insincere))
sns.barplot(x=y,y=x)

Pentagram sincere questions

In [ ]:
pentagram_sincere=get_top_ngram(sincere_label,n=5)
x,y=map(list,zip(*pentagram_sincere))
sns.barplot(x=y,y=x)

# Data Cleaning

Getting rid of 
1. HTML codes
2. URLs
3. Emojis
4. Stopwords
5. Punctuations
6. Expanding Abbreviations

In [ ]:
%%time
import re
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

Removing punctuations

In [ ]:
train['question_text']=train['question_text'].apply(lambda z: remove_punctuations(z))

In [ ]:
#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

In [ ]:
train['question_text']=train['question_text'].apply(lambda z: remove_html(z))

Removing URL

In [ ]:
#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

In [ ]:
train['question_text']=train['question_text'].apply(lambda z: remove_url(z))

Removing Emojis

In [ ]:
#Removes Emojis
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

In [ ]:
train['question_text']=train['question_text'].apply(lambda z: remove_emoji(z))

In [ ]:
train['question_text'][:5]

# Transforming corpus

Lemmantization

In [ ]:
#Lemmatize the dataset
from nltk.stem import WordNetLemmatizer


def lemma_traincorpus(data):
    lemmatizer=WordNetLemmatizer()
    out_data=""
    for words in data:
        out_data+= lemmatizer.lemmatize(words)
    return out_data

train['question_text']=train['question_text'].apply(lambda z: lemma_traincorpus(z))

train['question_text'].sample()

Stemming

In [ ]:
from nltk.stem import *

def stem_traincorpus(data):
    stemmer = PorterStemmer()
    out_data=""
    for words in data:
        out_data+= stemmer.stem(words)
    return out_data

train['question_text']=train['question_text'].apply(lambda z: stem_traincorpus(z))

train['question_text'].sample()

# Vectorization and Embeddings

## TF-IDF

In [ ]:
%%time
tfidf_vect=TfidfVectorizer(stop_words='english',
                           ngram_range=(1,3))

train_tfidf=tfidf_vect.fit_transform(train['question_text'].values.tolist())

train_tfidf.shape

In [ ]:
print(train_tfidf)

## Visualizing the Vector Space

In [ ]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np 

In [ ]:
#Count Vectorization
import matplotlib
import matplotlib.pyplot as plt

def vectorize(data):
    cv=CountVectorizer()
    fit_data_cv=cv.fit_transform(data)
    return fit_data_cv,cv

In [ ]:
#Tfidf vectorization from sklearn
def tfidf(data):
    tfidfv=TfidfVectorizer()
    fit_data_tfidf=tfidfv.fit_transform(data)
    return fit_data_cv,tfidfv

## Dimensionality reduction

In [ ]:
def dimen_reduc_plot(test_data,
                     test_label,
                     option):
    tsvd= TruncatedSVD(n_components=2,
                       algorithm="randomized",
                       random_state=42)
    tsne=TSNE(n_components=2,
              random_state=42) #not recommended instead use PCA
    pca=SparsePCA(n_components=2,
                  random_state=42)
    if(option==1):
        tsvd_result=tsvd.fit_transform(test_data)
        plt.figure(figsize=(10,8))
        colors=['orange','red']
        
        sns.scatterplot(x=tsvd_result[:,0],
                        y=tsvd_result[:,1],
                        hue=test_label        )
        
        plt.show()
        plt.figure(figsize=(10,10))
        plt.scatter(tsvd_result[:,0],
                    tsvd_result[:,1],
                    c=test_label,
                    cmap=matplotlib.colors.ListedColormap(colors))
        color_red=mpatches.Patch(color='red',
                                 label='Insincere Questions')
        color_orange=mpatches.Patch(color='orange',
                                    label='Sincere Questions')
        plt.legend(handles=[color_orange,
                            color_red])
        plt.title("TSVD")
        plt.show()
    if(option==2):
        tsne_result=tsne.fit_transform(test_data)
        plt.figure(figsize=(10,8))
        colors=['orange','red']
        sns.scatterplot(x=tsne_result[:,0],
                        y=tsne_result[:,1],
                        hue=test_label)
        plt.show()
        plt.figure(figsize=(10,10))
        plt.scatter(x=tsne_result[:,0],
                    y=tsne_result[:,1],
                    c=test_label,
                    cmap=matplotlib.colors.ListedColormap(colors))
        color_red=mpatches.Patch(color='red',
                                 label='Insincere Questions')
        color_orange=mpatches.Patch(color='orange',
                                    label='Sincere Questions')
        plt.legend(handles=[color_orange,
                            color_red])
        plt.title("PCA")
        plt.show() 
    if(option==3):
        pca_result=pca.fit_transform(test_data.toarray())
        plt.figure(figsize=(10,8))
        colors=['orange','red']
        sns.scatterplot(x=pca_result[:,0],
                        y=pca_result[:,1],
                        hue=test_label)
        plt.show()
        plt.figure(figsize=(10,10))
        plt.scatter(x=pca_result[:,0],
                    y=pca_result[:,1],
                    c=test_label,
                    cmap=matplotlib.colors.ListedColormap(colors))
        color_red=mpatches.Patch(color='red',label='Insincere Questions')
        color_orange=mpatches.Patch(color='orange',
                                    label='Sincere Questions')
        plt.legend(handles=[color_orange,color_red])
        plt.title("TSNE")
        plt.show()

In [ ]:
train_data=train       

data_vect=train_data['question_text'].values

data_vect_good=count_good['question_text'].values

target_vect=train_data['target'].values

target_data_vect_good=train[train['target']==0].values

data_vect_bad=count_bad['question_text'].values

target_data_vect_bad=train[train['target']==1].values

train_data_cv,cv= vectorize(data_vect)

real_review_train_data_cv,cv=vectorize(data_vect_good)

print(train_data.head())

dimen_reduc_plot(train_data_cv,target_vect,1)

dimen_reduc_plot(real_review_train_data_cv,target_data_vect_good,1)

dimen_reduc_plot(real_review_train_data_cv,target_data_vect_bad,1)

# dimen_reduc_plot(train_data_cv,target_vect,3)
# dimen_reduc_plot(real_review_train_data_cv,target_data_vect_good,3)
# dimen_reduc_plot(train_data_cv,target_vect,2)
# dimen_reduc_plot(real_review_train_data_cv,target_data_vect_good,2)

## TSNE visualization

In [ ]:
#TSNE visualization on first 1000 samples
train_data=train[:1000]       

data_vect=train_data['question_text'].values

data_vect_good=count_good['question_text'].values

target_vect=train_data['target'].values

target_data_vect_good=train[train['target']==1].values

data_vect_bad=count_bad['question_text'].values

target_data_vect_bad=train[train['target']==1].values

train_data_cv,cv= vectorize(data_vect)

real_review_train_data_cv,cv=vectorize(data_vect_good)

dimen_reduc_plot(train_data_cv,target_vect,3)

In [ ]:
check_df=list(train['question_text'].str.split())

# Semantic Embeddings

In [ ]:
%%time
## Load word2vec algorithm from gensim
from gensim.models import Word2Vec,KeyedVectors

In [ ]:
model=Word2Vec(check_df,min_count=1)
word_li=list(model.wv.vocab)
print(word_li[:5])

In [ ]:
#View the Tensor
print(model)
print(model['questions'])

In [ ]:
##save the modeled words produced from Word2Vec
model.save('word2vec_model.bin')
loaded_model=KeyedVectors.load('word2vec_model.bin')
print(loaded_model)

In [ ]:
#View the Embedding Word Vector
plt.plot(model['questions'])
plt.show()

# Dimension reduction of the embedding vectors

In [ ]:
train.sample()

In [ ]:
#Measure Cosine distance
distance=model.similarity('questions','insincere')
print(distance)

In [ ]:
# PCA transform in 2D for visualization of embedded words
from matplotlib import pyplot

pca = PCA(n_components=2)

transformation_model=loaded_model[loaded_model.wv.vocab]

result = pca.fit_transform(transformation_model[:50])

In [ ]:
# create a scatter plot of the projection

pyplot.scatter(result[:, 0], result[:, 1])
words = list(loaded_model.wv.vocab)
for i, word in enumerate(words[:50]):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

In [ ]:
#Using Google News Embeddings For our corpus
google_news_embed='../input/nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin'
google_loaded_model=KeyedVectors.load_word2vec_format(google_news_embed,binary=True)
print(google_loaded_model)

In [ ]:
#Visualize the Word Vectors
plt.plot(google_loaded_model['questions'])
plt.plot(google_loaded_model['insincere'])
plt.show()

## PCA transform in 2D for visualization of google news embedded words

In [ ]:
from matplotlib import pyplot

In [ ]:
pca = PCA(n_components=2)

transformation_model=google_loaded_model[google_loaded_model.wv.vocab]

result = pca.fit_transform(transformation_model[:50])

# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])

words = list(google_loaded_model.wv.vocab)

for i, word in enumerate(words[:50]):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

   
# Working with Glove Embeddings

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file='../input/nlpword2vecembeddingspretrained/glove.6B.50d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove_loaded=glove2word2vec(glove_file, word2vec_output_file)
print(glove_loaded)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
plt.plot(glove_model['questions'])
plt.plot(glove_model['insincere'])
plt.show()

In [ ]:
# PCA transform in 2D for visualization of glove embedded words
from matplotlib import pyplot
pca = PCA(n_components=2)

transformation_model=glove_model[glove_model.wv.vocab]
result = pca.fit_transform(transformation_model[:50])

# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(glove_model.wv.vocab)
for i, word in enumerate(words[:50]):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

## Using Fasttext

In [ ]:
from gensim.models import Word2Vec,KeyedVectors

fasttext_file="../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec"
print(fasttext_file)

In [ ]:
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_file, binary=False)
plt.plot(fasttext_model['questions'])
plt.plot(fasttext_model['insincere'])
plt.show()

In [ ]:
# PCA transform in 2D for visualization of glove embedded words
from matplotlib import pyplot
pca = PCA(n_components=2)

transformation_model=fasttext_model[fasttext_model.wv.vocab]
result = pca.fit_transform(transformation_model[:50])

# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(fasttext_model.wv.vocab)
for i, word in enumerate(words[:50]):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

## Using Keras

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [ ]:
#Creating Embedding Matrix
maxlen=1000
max_features=5000
embed_size=300

train_sample=train['question_text']

In [ ]:
#Tokenizing steps- must be remembered
tokenizer=Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_sample))
train_sample=tokenizer.texts_to_sequences(train_sample)

In [ ]:
#Pad the sequence- To allow same length for all vectorized words
train_sample=pad_sequences(train_sample,maxlen=maxlen)

EMBEDDING_FILE = '../input/wikinews300d1msubwordvec/wiki-news-300d-1M-subword.vec'

In [ ]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
plt.plot(embedding_matrix[20])
plt.show()

## Compeletion of static embeddings